# Use Openbabel to create features

In [1]:
# install openbabel
!conda install -c openbabel openbabel -y

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.0=py37_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37hdd07704_0
  - defaults/linux-64::pandas==0.23.4=py37h04863e7_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::h5py==2.8.0=py37h989c5e5_3
  - defaults/linux-64::numpy-base==1.15.1=py37h81de0dd_0
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::scikit-learn==0.19.2=py37h4989274_0
  - defaults/linux-64::statsmodels==0.9.0=py37h035aef0_0
  - defaults/linux-64::bkcharts==

In [2]:
import pandas as pd
import numpy as np
import os
import csv   
import openbabel
from tqdm import tqdm
obConversion = openbabel.OBConversion()
_ = obConversion.SetInAndOutFormats("xyz", "mol2")

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')
structures = pd.read_csv('../input/structures.csv')
scalar_coupling_contributions = pd.read_csv('../input/scalar_coupling_contributions.csv')

In [4]:
count = 0

with open(r'molecule_details.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', 'mol_wt', 'num_atoms','num_bonds','num_residues'])
with open(r'bond_details.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', 'left_atom_idx', 'right_atom_idx','equib_length','bond_length','BO','Flags'])
with open(r'angle_details.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', 'left_atom_idx', 'middle_atom_idx','right_atom_idx','angle'])
with open(r'atom_details.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', 'atom_idx', 'atomic_mass','atomic_number','exact_mass','valence','spin_multiplicity'])
with open(r'torsion_details.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', '2left_atom_idx', 'left_atom_idx','right_atom_idx','2right_atom_idx','torsion_angle'])
with open(r'distances.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', 'left_atom_idx', 'right_atom_idx','distance','is_bond'])
with open(r'closest.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['molecule_name', 'atom_index', 'closest','2nd_closest','3rd_closest','4th_closest','5th_closest','6th_closest',
                     '7th_closest','8th_closest','9th_closest','10th_closest','11th_closest','12th_closest','13th_closest','14th_closest',
                    '15th_closest','16th_closest','17th_closest','18th_closest','19th_closest','20th_closest','21st_closest','22nd_closest',
                    '23rd_closest','24th_closest','25th_closest','26th_closest','27th_closest','28th_closest'])

for xyz in tqdm(os.listdir('../input/structures/')):
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, '../input/structures/{}'.format(xyz))
    mol_name = xyz.split('.')[0]
#     print(xyz)
#     ### Loop through atoms
#     print('=======Molecule Details=======')
#     print('Molecule Weight: {}'.format(mol.GetMolWt()))
#     print('Number of Atoms: {}'.format(mol.NumAtoms()))
#     print('Number of Bonds: {}'.format(mol.NumBonds()))
#     print('Number of Res: {}'.format(mol.NumResidues()))    
#     ### Loop through bonds
#     print('=======Bond Details=======')
#     print(['molecule_name', 'left_atom_idx', 'right_atom_idx','equib_length','bond_length','BO','Flags'])
    with open(r'molecule_details.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([mol_name, mol.GetMolWt(), mol.NumAtoms(), mol.NumBonds(), mol.NumResidues()])
    for bond in openbabel.OBMolBondIter(mol):
#         print(mol_name, bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond.GetEquibLength(), bond.GetLength(),  bond.GetBO(), bond.GetFlags())
#         print(mol_name, bond.GetEndAtomIdx(), bond.GetBeginAtomIdx(), bond.GetEquibLength(), bond.GetLength(),  bond.GetBO(), bond.GetFlags())
        with open(r'bond_details.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([mol_name, bond.GetBeginAtomIdx()-1, bond.GetEndAtomIdx()-1, bond.GetEquibLength(), bond.GetLength(),  bond.GetBO(), bond.GetFlags()])
            writer.writerow([mol_name, bond.GetEndAtomIdx()-1, bond.GetBeginAtomIdx()-1, bond.GetEquibLength(), bond.GetLength(),  bond.GetBO(), bond.GetFlags()])
#     print('=======Angle Details=======')
#     print('molecule_name', 'left_atom_idx', 'middle_atom_idx','right_atom_idx','angle')
    for angle in openbabel.OBMolAngleIter(mol):
        a = (angle[0] + 1)
        b = mol.GetAtom(angle[1] + 1)
        c = (angle[2] + 1)
#         print(mol_name,a-1,b.GetIdx()-1,c-1, b.GetAngle(a, c))
#         print(mol_name,c-1,b.GetIdx()-1,a-1, b.GetAngle(c, a))
        with open(r'angle_details.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([mol_name,a-1,b.GetIdx()-1,c-1, b.GetAngle(a, c)])
            writer.writerow([mol_name,c-1,b.GetIdx()-1,a-1, b.GetAngle(c, a)])
#     print('=======Atom Details=======')
#     print('molecule_name', 'atom_idx', 'atomic_mass','atomic_number','exact_mass','valence','spin_multiplicity')
    for atom_no in range(1, mol.NumAtoms()+1):
        atom = mol.GetAtom(atom_no)
#         print(mol_name, atom.GetIdx()-1, atom.GetAtomicMass(), atom.GetAtomicNum(), atom.GetExactMass(), atom.GetValence(), atom.GetSpinMultiplicity())
        with open(r'atom_details.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([mol_name, atom.GetIdx()-1, atom.GetAtomicMass(), atom.GetAtomicNum(),
                             atom.GetExactMass(), atom.GetValence(), atom.GetSpinMultiplicity()])
#     print('=======Torsion Details=======')
#     print('molecule_name', '2left_atom_idx', 'left_atom_idx','right_atom_idx','2right_atom_idx','torsion_angle')
    for torsion in openbabel.OBMolTorsionIter(mol):
        a = (torsion[0] + 1)
        b = (torsion[1] + 1)
        c = (torsion[2] + 1)
        d = (torsion[3] + 1)
#         print(mol_name, a-1, b-1, c-1, d-1, mol.GetTorsion(a, b, c, d))
#         print(mol_name, d-1, c-1, b-1, a-1, mol.GetTorsion(d, c, b, a))
        with open(r'torsion_details.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([mol_name, a-1, b-1, c-1, d-1, mol.GetTorsion(a, b, c, d)])
            writer.writerow([mol_name, d-1, c-1, b-1, a-1, mol.GetTorsion(d, c, b, a)])
    ### DISTANCES
    for atom_left in range(1, mol.NumAtoms()+1):
        atoml = mol.GetAtom(atom_left)
        atom_dict = {}
        for atom_right in range(1, mol.NumAtoms()+1):
            
            atomr = mol.GetAtom(atom_right)
            is_bond = mol.GetBond(atoml, atomr)
            if is_bond is None:
                is_bond = False
            else:
                is_bond = True
            with open(r'distances.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([mol_name, atom_left-1, atom_right-1, atoml.GetDistance(atomr), is_bond])
            atom_dict[atom_right] = atoml.GetDistance(atomr)
        closest_first_dist = [k for k in sorted(atom_dict, key=atom_dict.get)]
        with open(r'closest.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([mol_name, atom_left] + closest_first_dist[1:])
#     if count == 500:
#         break
#     count += 1

 33%|███▎      | 43011/130775 [13:51<28:03, 52.14it/s]

In [5]:
bond_df = pd.read_csv('bond_details.csv')
angle_df = pd.read_csv('angle_details.csv')
atom_df = pd.read_csv('atom_details.csv')
tortion_df = pd.read_csv('torsion_details.csv')
distances_df = pd.read_csv('distances.csv')
closest_df = pd.read_csv('closest.csv')

In [6]:
distances_df.head()

,molecule_name,left_atom_idx,right_atom_idx,distance,is_bond
0,dsgdb9nsd_033805,0,0,0.000000,False
1,dsgdb9nsd_033805,0,1,1.155785,True
2,dsgdb9nsd_033805,0,2,2.606947,False
3,dsgdb9nsd_033805,0,3,3.616122,False
4,dsgdb9nsd_033805,0,4,4.788397,False


In [7]:
# Merge with distances - train set
pd.merge(train.sample(10, random_state=529),
         distances_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','right_atom_idx']) \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,distance,is_bond
0,3758951,dsgdb9nsd_107248,14,1,3JHC,3.655620,2.671750,False
1,2900578,dsgdb9nsd_085883,9,1,2JHN,1.558350,2.087678,False
2,4049013,dsgdb9nsd_114046,12,3,2JHC,7.772570,2.338395,False
3,915315,dsgdb9nsd_032217,9,10,2JHH,-11.760300,1.764299,False
4,740354,dsgdb9nsd_024302,10,7,3JHC,0.360284,3.049497,False
5,3017217,dsgdb9nsd_088745,13,6,3JHC,-0.194026,3.004576,False
6,12939,dsgdb9nsd_000640,7,5,3JHC,5.560950,2.985885,False
7,2845584,dsgdb9nsd_084421,14,4,3JHC,1.494210,2.570868,False
8,1610072,dsgdb9nsd_052244,12,1,3JHC,-0.123839,2.725683,False
9,3375778,dsgdb9nsd_097259,15,7,1JHN,57.920700,1.006487,True


In [8]:
# Merge with distances - test set
pd.merge(test.sample(10, random_state=529),
         distances_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','right_atom_idx']) \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

,id,molecule_name,atom_index_0,atom_index_1,type,distance,is_bond
0,5355513,dsgdb9nsd_042019,11,1,1JHC,1.096935,True
1,4700032,dsgdb9nsd_002879,9,4,2JHN,2.113530,False
2,6974321,dsgdb9nsd_119906,15,3,2JHC,2.145142,False
3,4747940,dsgdb9nsd_006507,15,1,2JHC,2.240096,False
4,5810170,dsgdb9nsd_066855,14,3,3JHC,3.567916,False
5,6901208,dsgdb9nsd_116506,19,21,3JHH,3.070922,False
6,5922072,dsgdb9nsd_072119,16,6,1JHC,1.089958,True
7,5602100,dsgdb9nsd_057188,9,10,2JHH,1.773062,False
8,6918912,dsgdb9nsd_117461,17,18,3JHH,2.266473,False
9,5513528,dsgdb9nsd_051570,10,2,1JHC,1.096327,True


In [9]:
angle_df.head()

,molecule_name,left_atom_idx,middle_atom_idx,right_atom_idx,angle
0,dsgdb9nsd_033805,1,0,2,179.876516
1,dsgdb9nsd_033805,2,0,1,0.054746
2,dsgdb9nsd_033805,2,1,8,116.116170
3,dsgdb9nsd_033805,8,1,2,30.742854
4,dsgdb9nsd_033805,2,6,8,107.535858


In [10]:
# Test merge on bond angles left/middle
pd.merge(train.sample(10, random_state=529),
         angle_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','middle_atom_idx'])

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,left_atom_idx,middle_atom_idx,right_atom_idx,angle
0,3758951,dsgdb9nsd_107248,14,1,3JHC,3.655620,NaN,NaN,NaN,NaN
1,2900578,dsgdb9nsd_085883,9,1,2JHN,1.558350,9.0,1.0,0.0,40.719294
2,4049013,dsgdb9nsd_114046,12,3,2JHC,7.772570,12.0,3.0,2.0,32.931255
3,915315,dsgdb9nsd_032217,9,10,2JHH,-11.760300,NaN,NaN,NaN,NaN
4,740354,dsgdb9nsd_024302,10,7,3JHC,0.360284,NaN,NaN,NaN,NaN
5,3017217,dsgdb9nsd_088745,13,6,3JHC,-0.194026,NaN,NaN,NaN,NaN
6,12939,dsgdb9nsd_000640,7,5,3JHC,5.560950,NaN,NaN,NaN,NaN
7,2845584,dsgdb9nsd_084421,14,4,3JHC,1.494210,NaN,NaN,NaN,NaN
8,1610072,dsgdb9nsd_052244,12,1,3JHC,-0.123839,NaN,NaN,NaN,NaN
9,3375778,dsgdb9nsd_097259,15,7,1JHN,57.920700,NaN,NaN,NaN,NaN


In [11]:
# Test merge on bond angles right/middle
pd.merge(train.sample(10, random_state=529),
         angle_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','right_atom_idx','middle_atom_idx'])

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,left_atom_idx,middle_atom_idx,right_atom_idx,angle
0,3758951,dsgdb9nsd_107248,14,1,3JHC,3.655620,NaN,NaN,NaN,NaN
1,2900578,dsgdb9nsd_085883,9,1,2JHN,1.558350,0.0,1.0,9.0,109.703217
2,4049013,dsgdb9nsd_114046,12,3,2JHC,7.772570,2.0,3.0,12.0,124.571782
3,915315,dsgdb9nsd_032217,9,10,2JHH,-11.760300,NaN,NaN,NaN,NaN
4,740354,dsgdb9nsd_024302,10,7,3JHC,0.360284,NaN,NaN,NaN,NaN
5,3017217,dsgdb9nsd_088745,13,6,3JHC,-0.194026,NaN,NaN,NaN,NaN
6,12939,dsgdb9nsd_000640,7,5,3JHC,5.560950,NaN,NaN,NaN,NaN
7,2845584,dsgdb9nsd_084421,14,4,3JHC,1.494210,NaN,NaN,NaN,NaN
8,1610072,dsgdb9nsd_052244,12,1,3JHC,-0.123839,NaN,NaN,NaN,NaN
9,3375778,dsgdb9nsd_097259,15,7,1JHN,57.920700,NaN,NaN,NaN,NaN


In [12]:
tortion_df.loc[tortion_df['molecule_name'] == 'dsgdb9nsd_032370'].sort_values('left_atom_idx')

,molecule_name,2left_atom_idx,left_atom_idx,right_atom_idx,2right_atom_idx,torsion_angle
1034198,dsgdb9nsd_032370,9,0,1,2,73.950769
1034202,dsgdb9nsd_032370,9,0,1,11,-49.693965
1034200,dsgdb9nsd_032370,9,0,1,10,-165.810322
1034213,dsgdb9nsd_032370,0,1,2,3,11.241515
1034217,dsgdb9nsd_032370,11,1,2,3,137.020293
1034219,dsgdb9nsd_032370,0,1,2,8,-169.507297
1034221,dsgdb9nsd_032370,10,1,2,8,72.172787
1034223,dsgdb9nsd_032370,11,1,2,8,-43.728519
1034203,dsgdb9nsd_032370,11,1,0,9,-49.693965
1034201,dsgdb9nsd_032370,10,1,0,9,-165.810322


In [13]:
train.loc[train['molecule_name'] == 'dsgdb9nsd_032370']

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
918998,918998,dsgdb9nsd_032370,9,1,2JHC,-1.092240
918999,918999,dsgdb9nsd_032370,9,2,3JHC,-0.569734
919000,919000,dsgdb9nsd_032370,9,10,3JHH,12.712400
919001,919001,dsgdb9nsd_032370,9,11,3JHH,4.045410
919002,919002,dsgdb9nsd_032370,10,1,1JHC,93.997900
919003,919003,dsgdb9nsd_032370,10,2,2JHC,-5.048780
919004,919004,dsgdb9nsd_032370,10,3,3JHC,3.873770
919005,919005,dsgdb9nsd_032370,10,8,3JHN,0.057257
919006,919006,dsgdb9nsd_032370,10,11,2JHH,-15.129400
919007,919007,dsgdb9nsd_032370,11,1,1JHC,95.563500


In [14]:
# merge with tortion
pd.merge(train.sample(10, random_state=529),
         tortion_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','right_atom_idx'])

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,2left_atom_idx,left_atom_idx,right_atom_idx,2right_atom_idx,torsion_angle
0,3758951,dsgdb9nsd_107248,14,1,3JHC,3.655620,NaN,NaN,NaN,NaN,NaN
1,2900578,dsgdb9nsd_085883,9,1,2JHN,1.558350,9.0,0.0,1.0,2.0,175.044573
2,2900578,dsgdb9nsd_085883,9,1,2JHN,1.558350,9.0,0.0,1.0,6.0,65.699532
3,4049013,dsgdb9nsd_114046,12,3,2JHC,7.772570,12.0,2.0,3.0,4.0,86.888484
4,4049013,dsgdb9nsd_114046,12,3,2JHC,7.772570,12.0,2.0,3.0,13.0,-36.760191
5,4049013,dsgdb9nsd_114046,12,3,2JHC,7.772570,12.0,2.0,3.0,6.0,-146.092219
6,915315,dsgdb9nsd_032217,9,10,2JHH,-11.760300,NaN,NaN,NaN,NaN,NaN
7,740354,dsgdb9nsd_024302,10,7,3JHC,0.360284,NaN,NaN,NaN,NaN,NaN
8,3017217,dsgdb9nsd_088745,13,6,3JHC,-0.194026,NaN,NaN,NaN,NaN,NaN
9,12939,dsgdb9nsd_000640,7,5,3JHC,5.560950,NaN,NaN,NaN,NaN,NaN


In [15]:
# merge with tortion
pd.merge(train.sample(50, random_state=529),
         tortion_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','2right_atom_idx']).dropna()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,2left_atom_idx,left_atom_idx,right_atom_idx,2right_atom_idx,torsion_angle


In [16]:
# merge with tortion
pd.merge(train.sample(10, random_state=529),
         tortion_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','left_atom_idx']).dropna()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,2left_atom_idx,left_atom_idx,right_atom_idx,2right_atom_idx,torsion_angle
9,3375778,dsgdb9nsd_097259,15,7,1JHN,57.9207,15.0,7.0,6.0,8.0,-2.381447
10,3375778,dsgdb9nsd_097259,15,7,1JHN,57.9207,15.0,7.0,6.0,3.0,178.301905


In [17]:
# merge with tortion
pd.merge(train.sample(10, random_state=529),
         tortion_df,
         how='left',
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2right_atom_idx','right_atom_idx']).dropna()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,2left_atom_idx,left_atom_idx,right_atom_idx,2right_atom_idx,torsion_angle
9,3375778,dsgdb9nsd_097259,15,7,1JHN,57.9207,8.0,6.0,7.0,15.0,-2.381447
10,3375778,dsgdb9nsd_097259,15,7,1JHN,57.9207,3.0,6.0,7.0,15.0,178.301905


# Find Closest Atom To Each Other Atom

In [18]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [19]:
distances_df[(distances_df['molecule_name'] == 'dsgdb9nsd_000001') &
             (distances_df['left_atom_idx'] == 2) &
             (distances_df['left_atom_idx'] != distances_df['right_atom_idx'])].sort_values('distance').T

,14971953,14971954,14971957,14971956
molecule_name,dsgdb9nsd_000001,dsgdb9nsd_000001,dsgdb9nsd_000001,dsgdb9nsd_000001
left_atom_idx,2,2,2,2
right_atom_idx,0,1,4,3
distance,1.09195,1.78312,1.78315,1.78316
is_bond,True,False,False,False


In [20]:
d = {}
d['1'] = 1
d['5'] = 18
d['3'] = 3
d

{'1': 1, '5': 18, '3': 3}

In [21]:
{k: d[k] for k in sorted(d, key=d.get)}

{'1': 1, '3': 3, '5': 18}

In [22]:
[k for k in sorted(d, key=d.get)]

['1', '3', '5']